In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("cdc/behavioral-risk-factor-surveillance-system")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/cdc/behavioral-risk-factor-surveillance-system/versions/1


In [ ]:
!ls /root/.cache/kagglehub/datasets/cdc/behavioral-risk-factor-surveillance-system/versions/1

2011.csv  2012.csv  2013.csv  2014.csv	2015.csv  2015_formats.json


In [ ]:
import os

data = pd.read_csv(os.path.join(path, '2015.csv'))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import copy

data_copy = copy.deepcopy(data)
columns_with_missing = ['_BMI5', 'PAFREQ1_', 'PAFREQ2_']

for column in columns_with_missing:
    data_with_values = data_copy.dropna(subset=[column])
    data_missing_values = data_copy[data_copy[column].isna()]
    if data_missing_values.empty:
        continue
    X = data_with_values.select_dtypes(include=[np.number]).drop(columns=columns_with_missing, errors='ignore').dropna()
    y = data_with_values.loc[X.index, column]
    if len(X) < 2:  # Ensure at least one train and one test sample
        continue
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    X_missing = data_missing_values.select_dtypes(include=[np.number]).drop(columns=columns_with_missing, errors='ignore').dropna()
    if not X_missing.empty:
        data_copy.loc[data_copy[column].isna(), column] = model.predict(X_missing)

print("Missing values have been filled.")


Missing values have been filled.


## BMI Statistics

In [ ]:
def print_statistics(x):
    print(f'Missing Values: {x.isna().sum()/len(x) * 100:.2f}%')
    x = x.dropna()
    print(f'min {x.min():.2f}')
    print(f'max {x.max():.2f}')
    print(f'std {x.std():.2f}')
    print(f'mean {x.mean():.2f}')
    print(f'median {x.median():.2f}')

    quantiles  = x.quantile([0, 0.25, 0.5, 0.75])

    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)

    IRQ = q3 - q1

    print(f'q1: {q1:.2f}')
    print(f'q3: {q3:.2f}')
    print(f'IRQ: {IRQ:.2f}')

In [ ]:
bmi = data['_BMI5'] / 100
bmi2 = data_copy['_BMI5'] / 100

In [ ]:
print_statistics(bmi)

Missing Values: 8.24%
min 12.02
max 99.95
std 6.65
mean 28.04
median 26.95
q1: 23.73
q3: 30.90
IRQ: 7.17


### Exercise

In [ ]:
def get_number_of_minutes_per_week(x):
    if x['EXERANY2'] == 1:
        if (x['PAFREQ1_'] == 99000) or (x['PAFREQ2_'] == 99000):
            return np.NaN

        minutes_first = (x['PAFREQ1_'] / 1000) * x['PADUR1_']
        minutes_second = (x['PAFREQ2_'] / 1000) * x['PADUR2_']

        if pd.isna(minutes_first):
            if pd.isna(minutes_second):
                return minutes_second
            else:
                return np.NaN
        else:
            if pd.isna(minutes_second):
                return minutes_first
            else:
                return minutes_first + minutes_second
    elif x['EXERANY2'] == 2:
        return 0
    else:
        return np.NaN

In [ ]:
exercise = data.apply(get_number_of_minutes_per_week, axis=1)


NameError: name 'data' is not defined

In [ ]:
exercise2 = data_copy.apply(get_number_of_minutes_per_week, axis=1)

In [ ]:
print_statistics(exercise)

NameError: name 'print_statistics' is not defined

In [ ]:
outlier_threhold = 7 * 60 * 12

In [ ]:
print(f'Above threhold: {(exercise > outlier_threhold).sum()}')
print(f'Above threshold: {(exercise2 > outlier_threhold).sum()}')

In [ ]:
print(f'Above Possible: {(exercise > 7*60*24).sum()}')
print(f'Above Possible: {(exercise2 > 7*60*24).sum()}')

In [ ]:
print_statistics(exercise[exercise <= outlier_threhold])
print_statistics(exercise2[exercise <= outlier_threhold])

In [ ]:
def plot(x, name, upper_lim=None, bins=100, units=None, unit=None):
    # Plot the histogram with improved style
    plt.figure(figsize=(10, 6))  # Adjust the figure size for better readability
    plt.hist(exercise.dropna(), bins=bins, color='skyblue', edgecolor='black', alpha=0.7)

    additional = f' (up to {upper_lim}{"" if units is None else " " + units})'
    plt.title(f'Distribution of {name}{additional}', fontsize=16, fontweight='bold')  # Add a title
    plt.xlabel(f'{name}{f" ({units})" if units is not None else ""}', fontsize=14)  # Label the x-axis
    plt.ylabel('Frequency', fontsize=14)  # Label the y-axis
    plt.grid(axis='y', linestyle='--', alpha=0.5)  # Add grid lines for reference
    plt.tight_layout()  # Adjust layout for better spacing
    if upper_lim is not None:
        plt.xlim((0, upper_lim))
    plt.show()

In [ ]:
plot(exercise, name='Weekly Physical Activity', units='minutes', bins=1000, upper_lim=1500)

In [ ]:
(exercise <= 1500).sum()/exercise.count() * 100
(exercise2 <= 1500).sum()/exercise2.count() * 100

In [ ]:
(exercise == 0).sum()/exercise.count()
(exercise2 == 0).sum()/exercise2.count()

In [ ]:
na_values = exercise.isna() | bmi.isna()
na_values2 = exercise2.isna() | bmi2.isna()

In [ ]:
exercise_no_na = exercise[~na_values]
bmi_no_na = bmi[~na_values]

exercise_no_na2 = exercise[~na_values2]
bmi_no_na = bmi2[~na_values2]

In [ ]:
import statsmodels.api as sm

def perform_regression(x, y):
    # Independent variable (X) and dependent variable (y)


    # Add a constant to the independent variable
    X = sm.add_constant(x)

    # Fit the OLS model with robust standard errors
    model = sm.OLS(y, X).fit()  # Specify robust covariance type
    print(model.summary())

    # Plot the data and the regression line
    plt.scatter(x, y, alpha=0.7, color='skyblue', edgecolor='black', linewidth=0.5, label='Data points')
    plt.plot(x, model.predict(X), color='red', label='Regression line')
    plt.title('Linear Regression: BMI vs. Physical Acitivty')
    plt.xlabel('Weekly Physical Acitivty (minutes)')
    plt.ylabel('BMI')
    plt.legend()
    plt.grid(alpha=0.5)
    plt.show()

In [ ]:
plt.scatter(x, y, alpha=0.7, color='skyblue', edgecolor='black', linewidth=0.5, label='Data points')
plt.plot(x, model.predict(X), color='red', label='Regression line')
plt.title('Linear Regression: BMI vs. Physical Acitivty')
plt.xlabel('Weekly Physical Acitivty (minutes)')
plt.ylabel('BMI')
plt.legend()
plt.grid(alpha=0.5)
plt.show()

In [ ]:
perform_regression(exercise_no_na, bmi_no_na)

In [ ]:
outliers_mask2 = exercise_no_na2 > 7 * 60 * 12

perform_regression(exercise_no_na2[~outliers_mask2], bmi_no_na2[~outliers_mask2])

In [ ]:
outliers_mask = exercise_no_na > 7 * 60 * 12

perform_regression(exercise_no_na[~outliers_mask], bmi_no_na[~outliers_mask])

In [ ]:
plt.scatter(exercise_no_na, bmi_no_na, alpha=0.7, color='skyblue', edgecolor='black', linewidth=0.5, label='Data points')
plt.plot([0, exercise_no_na.max()], [28.4118, 28.4118 - 0.0012 * exercise_no_na.max()], 'r', label='all data')
plt.plot([0, exercise_no_na.max()], [28.4728, 28.4728 - 0.0014 * exercise_no_na.max()], 'r--', label='no outlier')
plt.title('Linear Regression: BMI vs. Physical Acitivty (up to 1500 minutes)')
plt.xlabel('Weekly Physical Acitivty (minutes)')
plt.ylabel('BMI')
plt.legend()
plt.grid(alpha=0.5)
plt.xlim((0, 1500))
plt.show()

In [ ]:
small_df = pd.DataFrame({'bmi': bmi_no_na, 'exercise': exercise_no_na})

In [ ]:
small_df.head()

In [ ]:
sample = small_df.sample(n=100_000)

bmi_sample = sample['bmi']
exercise_sample = sample['exercise']

In [ ]:
plt.scatter(exercise_sample, bmi_sample)
plt.xlim((0, 3500))
plt.plot(exercise_sample, 28.3878 - 0.0008 * exercise_sample, 'r')
plt.plot(exercise_sample, 28.5865 - 0.0015 * exercise_sample, 'r')

In [ ]:
def has_heart_problems(x):
    if (x['CVDINFR4'] == 1) or (x['CVDCRHD4'] == 1):
        return 1
    elif (x['CVDINFR4'] == 2) and (x['CVDCRHD4'] == 2):
        return 0
    else:
        return np.NaN

In [ ]:
heart_attack = data.apply(has_heart_problems, axis=1)
bmi = data['_BMI5'] / 100

In [ ]:
bmi.head()

In [ ]:
def logistic_regression(x, y):
    na_values = x.isna() | y.isna()
    x = x[~na_values]
    y = y[~na_values]

    x = sm.add_constant(x)
    # Fit the logistic regression model
    model = sm.Logit(y, x)
    result = model.fit()

    # Print or return the result
    print(result.summary())  # For detailed output
    return result


In [ ]:
result = logistic_regression(bmi, heart_attack)

In [ ]:
result.llr

In [ ]:
from scipy.stats import chi2

p_value = chi2.sf(result.llr, 1)
p_value